## Logistic Regression - Numerical Features


In [ ]:
import time
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection
import sklearn.preprocessing as preproc
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/with_all_ft_clean.pkl")
df.head()

,texts,pct_change1,pct_change2,beta1,beta2,mkt_excess1,mkt_excess2,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,VIX,label1,label2,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0,Mkt_Cap
0,Item 2.02\nResults of Operations and Financial...,0.039772,0.018824,0.1235,0.12100,0.039502,0.017531,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,86355.36
1,Item 2.02 Results of Operations and Financial ...,0.005894,0.019647,0.9098,0.91085,-0.003435,0.005013,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,20.47,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,125102.08
2,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,-0.019268,-0.019268,0.8742,0.87420,-0.013888,-0.013888,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,19.07,neg,neg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,126812.46
3,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.077505,0.035918,0.8901,0.88485,0.076161,0.027620,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,139273.80
4,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.076013,0.076013,0.9225,0.92250,0.070040,0.070040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,13.51,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,144649.28


In [ ]:
def label1(row):
    if row["pct_change1"] > row["mkt_excess1"]:
        return "pos"
    else:
        return "neg"

def label2(row):
    if row["pct_change2"] > row["mkt_excess2"]:
        return "pos"
    else:
        return "neg"

def label3(row):
    if row["pct_change2"] > row["mkt_excess2"] + 0.005:
        return "positive"
    elif row["pct_change2"] < row["mkt_excess2"] - 0.005:
        return "negative"
    else:
        return "neutral"

In [ ]:
df["label1"] = df.apply(label1, axis = 1)
df["label2"] = df.apply(label2, axis = 1)
df["label3"] = df.apply(label3, axis = 1)

In [ ]:
# get column names of categorical features
item_list = [i for i in df.columns if not i.find("item")]
ind_list = [i for i in df.columns if not i.find("ind")]
shrcd_list = [i for i in df.columns if not i.find("SHRCD")]
exc_list = [i for i in df.columns if not i.find("EXCHCD")]

In [ ]:
# combine all feature column names
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

In [ ]:
# Separate into features, documents and label
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

X = df[cols]

y = df['label2']

In [ ]:
# Split into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.2)

In [ ]:
# standardize volatility index and market cap by mean and standard deviation (standardize train and test sets separately)
x_scaler = StandardScaler()
X_train["VIX"] = x_scaler.fit_transform(np.array(X_train["VIX"]).reshape(-1,1))
X_test["VIX"] = x_scaler.transform(np.array(X_test["VIX"]).reshape(-1,1))
X_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(X_train["Mkt_Cap"]).reshape(-1,1))
X_test["Mkt_Cap"] = x_scaler.transform(np.array(X_test["Mkt_Cap"]).reshape(-1,1))

In [ ]:
# define logistic regression classifier
def simple_logistic_classify(X_tr, y_tr, X_test, y_test, _C=1.0):
    model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with features', score)
    return model

In [ ]:
# fit the model to numerical features
model = simple_logistic_classify(X_train, y_train, X_test, y_test)

Test Score with features 0.60423865468786


In [ ]:
# Hyperparameter tuning

# grid search parameters
param_grid_ = [{'C': [1e-5, 1e-3, 1e-1, 1e0, 1e1, 1e2], 'penalty':["l1", "l2", "elasticnet"]}]

#create grid search with 5-fold cross validation
model_search = sklearn.model_selection.GridSearchCV(LogisticRegression(), cv=5, param_grid=param_grid_)

In [ ]:
model_search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid=[{'C': [1e-05, 0.001, 0.1, 1.0, 10.0, 100.0],
                          'penalty': ['l1', 'l2', 'elasticnet']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
model_search.best_estimator_

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model_search.best_score_

0.6003116868641701

In [ ]:
# get best models' predictions
pred = model_search.predict(X_test)

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         neg       0.56      0.18      0.27      1792
         pos       0.61      0.90      0.73      2549

    accuracy                           0.60      4341
   macro avg       0.58      0.54      0.50      4341
weighted avg       0.59      0.60      0.54      4341

